## DASHBOARD INTERACTIVO: ANÁLISIS DE TWEETS SOBRE DESASTRES NATURALES

In [7]:
import dash
from dash import dcc, html, dash_table, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

# Load the data
file_path = "/content/dataframe_api_historico-2.csv"
df = pd.read_csv(file_path)

# Filter non-null values
df_filtered = df.dropna(subset=["Model2"])
df_filtered = df_filtered[df_filtered["Model2"] != "not informative"]

# Disaster type count
desastres_count = df_filtered["Model2"].value_counts().reset_index()
desastres_count.columns = ["Tipo de Desastre", "Total"]

# Model3 Information Type Count
info_type_filtered = df.dropna(subset=["Model3"])
info_type_filtered = info_type_filtered[info_type_filtered["Model3"] != "not informative"]
info_type_count = info_type_filtered["Model3"].value_counts().reset_index()
info_type_count.columns = ["Categoría de Información", "Total"]

# Model1 Classification Count
model1_count = df.dropna(subset=["Model1"])["Model1"].value_counts().reset_index()
model1_count.columns = ["Clasificación", "Total"]

# Initialize Dash app with Bootstrap theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

# Layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Análisis de Tweets sobre Desastres Naturales", style={'color': 'black'}), width=12)
    ], className="mb-4 text-center"),

    dbc.Row([
        dbc.Col(html.H3(f"Total de Tweets analizados: {df.shape[0]}", style={'color': 'black'}), width=12)
    ], className="mb-3 text-center"),

    dbc.Row([
        dbc.Col(dcc.Dropdown(
            id='filter_model2',
            options=[{'label': tipo, 'value': tipo} for tipo in df_filtered["Model2"].unique()],
            placeholder="Filtrar por Tipo de Desastre",
            multi=True
        ), width=6),

        dbc.Col(dcc.Dropdown(
            id='filter_model3',
            options=[{'label': cat, 'value': cat} for cat in info_type_filtered["Model3"].unique()],
            placeholder="Filtrar por Categoría de Información",
            multi=True
        ), width=6),
    ], className="mb-4"),

    dbc.Row([
        dbc.Col(dcc.Graph(id="pie_model1", config={"responsive": True}), width=6),
        dbc.Col(dcc.Graph(id="grouped_bar_model2", config={"responsive": True}), width=6),
    ], className="mb-4"),

    dbc.Row([
        dbc.Col(dcc.Graph(id="grouped_bar_model3", config={"responsive": True}), width=6),
    ], className="mb-4"),

    dbc.Row([
        dbc.Col(html.H4("Total de Tweets por Clasificación", style={'color': 'black'}), width=6),
        dbc.Col(html.H4("Total de Desastres Naturales", style={'color': 'black'}), width=6),
    ], className="mb-2 text-center"),

    dbc.Row([
        dbc.Col(dash_table.DataTable(
            columns=[{"name": col, "id": col} for col in model1_count.columns],
            data=model1_count.to_dict("records"),
            page_size=5,
            style_table={'overflowX': 'auto'}
        ), width=6),

        dbc.Col(dash_table.DataTable(
            columns=[{"name": col, "id": col} for col in desastres_count.columns],
            data=desastres_count.to_dict("records"),
            page_size=5,
            style_table={'overflowX': 'auto'}
        ), width=6),
    ], className="mb-4"),

    dbc.Row([
        dbc.Col(html.H4("Tweets Clasificados", style={'color': 'black'}), width=12)
    ], className="mb-2 text-center"),

    dbc.Row([
        dbc.Col(dash_table.DataTable(
            id="filtered_table",
            columns=[{"name": col, "id": col} for col in ["CreatedAt", "Text", "Model1", "Model2", "Model3"]],
            data=df.to_dict("records"),
            page_size=10,
            style_table={'overflowX': 'auto'}
        ), width=12),
    ], className="mb-4"),
], fluid=True)


# Callbacks for interactive filtering
@app.callback(
    Output("grouped_bar_model2", "figure"),
    Output("grouped_bar_model3", "figure"),
    Output("pie_model1", "figure"),
    Output("filtered_table", "data"),
    Input("filter_model2", "value"),
    Input("filter_model3", "value")
)
def update_graphs(selected_model2, selected_model3):
    # Apply filters
    filtered_df = df_filtered.copy()
    if selected_model2:
        filtered_df = filtered_df[filtered_df["Model2"].isin(selected_model2)]
    if selected_model3:
        filtered_df = filtered_df[filtered_df["Model3"].isin(selected_model3)]

    # Update Disaster Type Bar Chart
    desastres_count_filtered = filtered_df["Model2"].value_counts().reset_index()
    desastres_count_filtered.columns = ["Tipo de Desastre", "Total"]

    grouped_bar_model2 = px.bar(
        desastres_count_filtered, x="Tipo de Desastre", y="Total",
        title="Distribución de Tipos de Desastres",
        labels={"Total": "Cantidad"},
        color="Tipo de Desastre",
        text_auto=True
    )

    # Update Model3 Category Bar Chart
    info_type_count_filtered = filtered_df["Model3"].value_counts().reset_index()
    info_type_count_filtered.columns = ["Categoría de Información", "Total"]

    grouped_bar_model3 = px.bar(
        info_type_count_filtered, x="Categoría de Información", y="Total",
        title="Distribución de Tipos de Información",
        labels={"Total": "Cantidad"},
        color="Categoría de Información",
        text_auto=True
    )

    # Update Model1 Pie Chart
    model1_count_filtered = filtered_df["Model1"].value_counts().reset_index()
    model1_count_filtered.columns = ["Clasificación", "Total"]

    pie_model1 = px.pie(
        model1_count_filtered, names="Clasificación", values="Total",
        title="Distribución de Informative vs Not Related"
    )

    return grouped_bar_model2, grouped_bar_model3, pie_model1, filtered_df.to_dict("records")


# Run server
if __name__ == "__main__":
    app.run_server(debug=True)


<IPython.core.display.Javascript object>